In [8]:
import json,os,sys,logging,pprint,re,string

In [79]:
nGramDict=json.load(open('/home/yewenhe0904/Developing/ds-capstone-project/ignoredFiles/Sample-Dict/twitter-nGramDict.json'))

In [80]:
nGramDict['_n']['W9']

{'_n': {'W10': {'_n': {'W1159': {'_r': ['W7']},
    'W21': {'_r': ['W15']},
    'W254': {'_r': ['W7']},
    'W32': {'_r': ['W1533']},
    'W83': {'_r': ['W7']}},
   '_r': ['W7', 'W254', 'W32', 'W83', 'W1159']},
  'W100': {'_r': ['W7']},
  'W1138': {'_n': {'W25': {'_r': ['W12']}}, '_r': ['W357', 'W7', 'W25']},
  'W1147': {'_r': ['W7', 'W405']},
  'W1159': {'_n': {'W282': {'_r': ['W7']}}, '_r': ['W282']},
  'W11784': {'_r': ['W7']},
  'W118': {'_r': ['W7']},
  'W12': {'_r': ['W7']},
  'W1373': {'_r': ['W32']},
  'W145': {'_r': ['W7']},
  'W15': {'_r': ['W206']},
  'W1514': {'_r': ['W206']},
  'W1652': {'_r': ['W7']},
  'W195': {'_n': {'W1315': {'_r': ['W69']}}, '_r': ['W1315', 'W7']},
  'W21': {'_r': ['W297']},
  'W231': {'_n': {'W11739': {'_r': ['W7']}}, '_r': ['W11739']},
  'W2465': {'_r': ['W7']},
  'W25': {'_n': {'W12': {'_r': ['W668']},
    'W370': {'_r': ['W69']},
    'W513': {'_r': ['W668']},
    'W86': {'_r': ['W1147']}},
   '_r': ['W513', 'W12', 'W2877', 'W86', 'W370']},
  'W254

In [81]:
test_input='he is donald'

In [82]:
def cleanInputPredict(test_input):
    cleanedInput=[]
    test_tokens=test_input.strip().lower().split()
    test_input=re.sub('\n+'," ",test_input)
    for item in test_tokens:
        item = item.strip(string.punctuation)
        item = re.sub(r"^.*\d+.*$","<Quantity>",item)
        cleanedInput.append(item)
    return cleanedInput

In [83]:
def predictCore(input_tokens,output_list,out_len=5):
    print (input_tokens)
    if len(input_tokens)<1:
        output_list+=(nGramDict['_r'])
    else:
        # probe if in ngram record
        nGram_flag=True
        i=0
        p_Dict=nGramDict['_n']
        while nGram_flag:
            print(i)
            if i==len(input_tokens):
                break
            if (input_tokens[i] in p_Dict):
                r_wids=p_Dict[input_tokens[i]]['_r']
                if ('_n' in p_Dict[input_tokens[i]]):
                    p_Dict=p_Dict[input_tokens[i]]['_n']
                    i+=1
                else:
                    if i<len(input_tokens)-1:
                        nGram_flag=False
                    break
            else:
                nGram_flag=False
                break
        # if true
        if nGram_flag:
            output_list+=r_wids
            if len(output_list)<out_len:
                del input_tokens[0]
                predictCore(input_tokens,output_list,out_len=out_len)
        # else false
        else:
            del input_tokens[0]
            predictCore(input_tokens,output_list,out_len=out_len)


        
        
        

In [84]:
def predictNgram(test_input,out_len):
    #clean input string
    input_tokens=cleanInputPredict(test_input)
    if len(input_tokens)>nGramDict['_model']:
        input_tokens=input_tokens[-nGramDict['_model']:]
    output_list=[]
    output_wlist=[]
    output=[]
    input_idtokens=[]
    for w in input_tokens:
        w.strip()
        if w in nGramDict['_word2id']['_word']:
            input_idtokens.append(nGramDict['_word2id']['_word'][w])
        else:
            input_idtokens.append('NA')
    predictCore(input_idtokens,output_list,out_len)
    for wid in output_list:
        output_wlist.append(nGramDict['_id2word']['_id'][wid])
    eosflag=True
    for word in output_wlist:
        if word=='<EOS>':
            if eosflag:
                output.append(',')
                output.append('.')
            eosflag=False
        elif word=="<Quantity>":
            pass
        else:
            output.append(word)
        #dedupe
        output_dp=[output[0]]
        if len(output)>1:
            for i in range(1,len(output)):
                if output[i] not in output[:i]:
                    output_dp.append(output[i])
        if len(output_dp)>out_len:
            output_dp=output_dp[0:out_len]
    return output_dp

In [85]:
predictNgram(test_input,8)

['W691', 'W24', 'W5751']
0
1
2
['W24', 'W5751']
0
1
['W5751']
0
[]


['trump', ',', '.', 'duck', 'driver', 'the', 'to', 'i']